In [2]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from datetime import datetime,timedelta

In [2]:
#read in the dataset
lst_data=Dataset(r'C:\Users\Mahe\Desktop\wildfire predictor\Datasets\MOD11A1.006_1km_aid0001b.nc','r')
ndvi_data=Dataset(r'C:\Users\Mahe\Desktop\wildfire predictor\Datasets\MOD13A2.006_1km_aid0001.nc','r')
vcr_data=Dataset(r'C:\Users\Mahe\Desktop\wildfire predictor\Datasets\MOD44B.006_250m_aid000199(2).nc','r')
prec_data=Dataset(r'C:\Users\Mahe\Desktop\wildfire predictor\Datasets\CDR_31_2020-06-24033126am_2019.nc','r')
ther_data=Dataset(r'C:\Users\Mahe\Desktop\wildfire predictor\Datasets\MOD14A2.006_1km_aid0001.nc','r')
#data.variables.keys()

In [3]:
#lst variables
lat=lst_data.variables['lat'][:]
lon=lst_data.variables['lon'][:]
lst=lst_data.variables['LST_Day_1km']
lstn=lst_data.variables['LST_Night_1km']

#ndvi variables
nd=ndvi_data.variables['_1_km_16_days_NDVI']
nlat=ndvi_data.variables['lat'][:]
nlon=ndvi_data.variables['lon'][:]

#vcr variables
vc=vcr_data.variables['Percent_NonTree_Vegetation'][:]
vlat=vcr_data.variables['lat'][:]
vlon=vcr_data.variables['lon'][:]

#prec variables
plat=prec_data.variables['lat'][:]
plon=prec_data.variables['lon'][:]
pre=prec_data.variables['precip']

#thermal anomalies variables
tlat=ther_data.variables['lat'][:]
tlon=ther_data.variables['lon'][:]
fm=ther_data.variables['FireMask']

In [4]:
tdr=[]
time=ther_data.variables['time'].units[11:21]
stt = datetime.strptime(time, "%Y-%m-%d")
for i in range(len(ther_data.variables['time'][:])):
    m=stt+timedelta(days=int(ther_data.variables['time'][i].data))
    tdr.append(m)
tdr.pop(0)    
td=[]
for i in range(len(tdr)):
    td.append(tdr[i].date().strftime('%Y-%m-%d'))
td  

['2019-06-02', '2019-06-10', '2019-06-18', '2019-06-26']

In [3]:
#import the frp dataset
df=pd.read_csv(r'C:\Users\Mahe\Desktop\wildfire predictor\loclist.csv')
df=df.drop(columns=['Unnamed: 0'])
#frp=frp[['latitude','longitude','acq_date','frp']]
#df=df[['latitude','longitude']]
df

,latitude,longitude
0,39.73886,-122.29385
1,39.73400,-122.29448
2,39.72914,-122.29508
3,39.73642,-122.29264
4,39.73156,-122.29321
...,...,...
1792,39.27449,-121.68446
1793,39.66578,-122.26375
1794,39.66512,-122.26955
1795,39.66611,-122.26579


In [13]:
column_names = ['date', 'lat', 'lon','mask']
frp= pd.DataFrame(columns = column_names)
frp

,date,lat,lon,mask


In [14]:
for index,row in df.iterrows():
    la=row['latitude']
    lo=row['longitude']
    sdflon=(tlon-lo)**2
    sdflat=(tlat-la)**2
    min_lat=sdflat.argmin()
    min_lon=sdflon.argmin()
    a=[0.0,0.0,0.0,0.0]
    a[1]=tlat[min_lat]
    a[2]=tlon[min_lon]
    tr=np.arange(1,len(tdr))
    for ti in tr:
        a[0]=td[ti-1]
        a[3]=fm[ti,min_lat,min_lon].__float__()
        #a[4]=lstn[ti,min_lat,min_lon].__float__()
        ase = pd.Series(a, index = frp.columns)
        frp=frp.append(ase,ignore_index=True)

C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Warning: converting a masked element to nan.
  


In [15]:
frp

,date,lat,lon,mask
0,2019-06-02,39.7375,-122.295833,9.0
1,2019-06-10,39.7375,-122.295833,5.0
2,2019-06-18,39.7375,-122.295833,5.0
3,2019-06-02,39.7375,-122.295833,9.0
4,2019-06-10,39.7375,-122.295833,5.0
...,...,...,...,...
5386,2019-06-10,39.6625,-122.262500,5.0
5387,2019-06-18,39.6625,-122.262500,8.0
5388,2019-06-02,39.6625,-122.270833,5.0
5389,2019-06-10,39.6625,-122.270833,5.0


In [16]:
frp=frp.drop_duplicates( keep='first')

In [17]:
frp

,date,lat,lon,mask
0,2019-06-02,39.737500,-122.295833,9.0
1,2019-06-10,39.737500,-122.295833,5.0
2,2019-06-18,39.737500,-122.295833,5.0
6,2019-06-02,39.729167,-122.295833,9.0
7,2019-06-10,39.729167,-122.295833,5.0
...,...,...,...,...
5362,2019-06-10,39.070833,-121.870833,5.0
5363,2019-06-18,39.070833,-121.870833,5.0
5373,2019-06-02,39.270833,-121.687500,5.0
5374,2019-06-10,39.270833,-121.687500,5.0


In [18]:
len(frp)

2181

In [19]:
#compute the date range for the time period of study

time=lst_data.variables['time'].units[11:21]
stt = datetime.strptime(time, "%Y-%m-%d")
stt=stt+timedelta(days=int(lst_data.variables['time'][0].data))
ett=stt+timedelta(days=30)
d_range=pd.date_range(start=stt,end=ett)
d=[]
for i in range(len(d_range)):
    d.append(d_range[i].date().strftime('%Y-%m-%d'))

In [20]:
d

['2019-06-01',
 '2019-06-02',
 '2019-06-03',
 '2019-06-04',
 '2019-06-05',
 '2019-06-06',
 '2019-06-07',
 '2019-06-08',
 '2019-06-09',
 '2019-06-10',
 '2019-06-11',
 '2019-06-12',
 '2019-06-13',
 '2019-06-14',
 '2019-06-15',
 '2019-06-16',
 '2019-06-17',
 '2019-06-18',
 '2019-06-19',
 '2019-06-20',
 '2019-06-21',
 '2019-06-22',
 '2019-06-23',
 '2019-06-24',
 '2019-06-25',
 '2019-06-26',
 '2019-06-27',
 '2019-06-28',
 '2019-06-29',
 '2019-06-30',
 '2019-07-01']

# Extract LST

In [21]:
ld=[]
ln=[]

In [22]:
for index,row in frp.iterrows():
    la=row['lat']
    lo=row['lon']
    sdflon=(lon-lo)**2
    sdflat=(lat-la)**2
    min_lat=sdflat.argmin()
    min_lon=sdflon.argmin()
    i=d.index(row['date'])
    ld.append(lst[i,min_lat,min_lon].__float__())
    ln.append(lstn[i,min_lat,min_lon].__float__())

C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Warning: converting a masked element to nan.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Warning: converting a masked element to nan.
  if __name__ == '__main__':


In [23]:
len(ld)

2181

In [24]:
la=39.737500
lo=-122.295833	
sdflon=(lon-lo)**2
sdflat=(lat-la)**2
min_lat=sdflat.argmin()
min_lon=sdflon.argmin()
x=lst[1,min_lat,min_lon].__float__()
x

310.26

# Extract NDVI

In [25]:
#to be changed based on time period of data/study
#list contains number of days between each reading
b=[9,16,6]
m=-1
x=[]
for i in b:
    m=m+1
    for n in range(i):
        x.append(m)
ndv=[]

In [26]:
for index,row in frp.iterrows():
    la=row['lat']
    lo=row['lon']
    sdflon=(nlon-lo)**2
    sdflat=(nlat-la)**2
    min_lat=sdflat.argmin()
    min_lon=sdflon.argmin()
    i=x[d.index(frp.loc[index,'date'])]
    ndv.append(nd[i,min_lat,min_lon].__float__())

C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Warning: converting a masked element to nan.
  if __name__ == '__main__':


In [27]:
len(ndv)

2181

# Extract VCR

In [28]:
vb=[]
for index,row in frp.iterrows():
    la=row['lat']
    lo=row['lon']
    sdflon=(vlon-lo)**2
    sdflat=(vlat-la)**2
    min_lat=sdflat.argmin()
    min_lon=sdflon.argmin()
    #i=d.index(row['acq_date'])
    vb.append(vc[0,min_lat,min_lon].__float__())
    #v=vc[0, min_lat, min_lat].__float__()

C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Warning: converting a masked element to nan.
  # Remove the CWD from sys.path while we load stuff.


In [29]:
len(vb)

2181

# Extract Precipitation values 

In [30]:
pr=[]

In [31]:
for index,row in frp.iterrows():
    la=row['lat']
    lo=row['lon']
    sdflon=(plon-lo)**2
    sdflat=(plat-la)**2
    min_lat=sdflat.argmin()
    min_lon=sdflon.argmin()
    i=d.index(row['date'])
    pr.append(pre[i,min_lat,min_lon].__float__())

In [32]:
len(pr)

2181

In [33]:
frp

,date,lat,lon,mask
0,2019-06-02,39.737500,-122.295833,9.0
1,2019-06-10,39.737500,-122.295833,5.0
2,2019-06-18,39.737500,-122.295833,5.0
6,2019-06-02,39.729167,-122.295833,9.0
7,2019-06-10,39.729167,-122.295833,5.0
...,...,...,...,...
5362,2019-06-10,39.070833,-121.870833,5.0
5363,2019-06-18,39.070833,-121.870833,5.0
5373,2019-06-02,39.270833,-121.687500,5.0
5374,2019-06-10,39.270833,-121.687500,5.0


In [34]:
# add columns to the frp dataframe
frp['lst']=ld
frp['lstn']=ln
frp['ndvi']=ndv
frp['vcr']=vb
frp['prec']=pr

C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Mahe\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [35]:
frp

,date,lat,lon,mask,lst,lstn,ndvi,vcr,prec
0,2019-06-02,39.737500,-122.295833,9.0,310.26,NaN,0.3108,59.0,0.718065
1,2019-06-10,39.737500,-122.295833,5.0,323.92,NaN,0.2526,59.0,0.000000
2,2019-06-18,39.737500,-122.295833,5.0,NaN,297.90,0.2526,59.0,0.000000
6,2019-06-02,39.729167,-122.295833,9.0,310.36,NaN,0.3198,57.0,0.718065
7,2019-06-10,39.729167,-122.295833,5.0,321.90,NaN,0.2670,57.0,0.000000
...,...,...,...,...,...,...,...,...,...
5362,2019-06-10,39.070833,-121.870833,5.0,308.10,288.26,0.4873,70.0,0.000000
5363,2019-06-18,39.070833,-121.870833,5.0,NaN,293.76,0.4873,70.0,0.000000
5373,2019-06-02,39.270833,-121.687500,5.0,303.80,NaN,0.5500,61.0,0.246932
5374,2019-06-10,39.270833,-121.687500,5.0,306.78,NaN,0.4881,61.0,0.000000


In [38]:
frp.to_csv(r'C:\Users\Mahe\Desktop\wildfire predictor\extract.csv')